# 📗 Notebook 01 – Image ETL from GCS and Preprocessing

## 🎯 Objective:
This notebook handles the first stage of the pipeline: **extracting raw images from Google Cloud Storage (GCS)**, preprocessing them, and saving them into a structured format.

We:
- Connect to GCS and list all images in the dataset
- Resize each image to `224x224` resolution for model compatibility
- Save the resized images back to GCS under a `/processed/` directory
- Extract and save basic metadata (filename, size, format) for each image
- Prepare data for the next step: emotion annotation

> This step ensures all images are standardized and ready for annotation and modeling.


*   Mục danh sách
*   Mục danh sách



In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=zIwX9pVUvgl9Rf4gHpmUCnumM4rKEn&prompt=consent&token_usage=remote&access_type=offline&code_challenge=C7mxWSxAAhDy9VJspeCwYAtX9Vuxm_TpybSERGHuAhY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQSTgQHg8nR1U3HrLzZFzzmK7NEdtOpZ9wl-ChptZbIHTNaWvKCPKQ2HvFjFCfXLylk-Iw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
from google.colab import auth
auth.authenticate_user()

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
import io
import os
from google.cloud import storage
from PIL import Image, UnidentifiedImageError

# Cấu hình thông tin dự án và bucket
project_id = 'exalted-summer-454012-d2'
bucket_name = 'boothill2001-dataset'
source_prefix = 'flickr_image_dataset/'
destination_prefix = 'flickr_image_dataset/processed_demo/'

# Tạo client và lấy bucket
client = storage.Client(project=project_id)
bucket = client.get_bucket(bucket_name)

# Lấy danh sách các file ảnh từ folder nguồn, giới hạn 10 mẫu
blobs = list(bucket.list_blobs(prefix=source_prefix))
image_blobs = [blob for blob in blobs if blob.name.lower().endswith(('.jpg', '.jpeg', '.png'))][:10]
print(f"Tìm thấy {len(image_blobs)} ảnh mẫu để demo.")

# Tạo folder đích trên GCS nếu cần (tùy thuộc vào cách bạn quản lý, thường chỉ cần upload vào destination_prefix)
for blob in image_blobs:
    try:
        # Tải dữ liệu ảnh từ GCS
        image_data = blob.download_as_bytes()
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
    except UnidentifiedImageError as e:
        print(f"Lỗi khi tải ảnh {blob.name}: {e}")
        continue

    # Resize ảnh về kích thước 224x224
    processed_image = image.resize((224, 224))

    # Lưu ảnh đã xử lý tạm thời lên local
    local_filename = os.path.basename(blob.name)
    processed_local_path = f"/tmp/processed_{local_filename}"
    processed_image.save(processed_local_path, format='JPEG')

    # Upload file đã xử lý lên folder destination trên GCS
    destination_blob_name = destination_prefix + local_filename
    destination_blob = bucket.blob(destination_blob_name)
    destination_blob.upload_from_filename(processed_local_path)
    print(f"Đã xử lý và upload {blob.name} lên {destination_blob_name}")

    # Xóa file tạm trên local
    os.remove(processed_local_path)

print("Quá trình ETL demo hoàn thành.")


/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Tìm thấy 10 ảnh mẫu để demo.
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000092795.jpg lên flickr_image_dataset/processed_demo/1000092795.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/10002456.jpg lên flickr_image_dataset/processed_demo/10002456.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000268201.jpg lên flickr_image_dataset/processed_demo/1000268201.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000344755.jpg lên flickr_image_dataset/processed_demo/1000344755.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000366164.jpg lên flickr_image_dataset/processed_demo/1000366164.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000523639.jpg lên flickr_image_dataset/processed_demo/1000523639.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/1000919630.jpg lên flickr_image_datas

In [5]:
import io
import os
from google.cloud import storage
from PIL import Image, UnidentifiedImageError

# Cấu hình thông tin dự án và bucket
project_id = 'exalted-summer-454012-d2'
bucket_name = 'boothill2001-dataset'
source_prefix = 'flickr_image_dataset/'
destination_prefix = 'flickr_image_dataset/processed/'

# Tạo client và lấy bucket
client = storage.Client(project=project_id)
bucket = client.get_bucket(bucket_name)

# Lấy danh sách các file trong folder nguồn
blobs = list(bucket.list_blobs(prefix=source_prefix))
print(f"Tìm thấy {len(blobs)} file trong folder nguồn.")

# Lặp qua các file ảnh và xử lý
for blob in blobs:
    # Chỉ xử lý các file có đuôi ảnh
    if not blob.name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    try:
        # Tải dữ liệu ảnh từ GCS
        image_data = blob.download_as_bytes()
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
    except UnidentifiedImageError as e:
        print(f"Lỗi khi tải ảnh {blob.name}: {e}")
        continue

    # Resize ảnh (ví dụ: 224x224)
    processed_image = image.resize((224, 224))

    # Lưu ảnh đã xử lý vào local (tạm thời)
    local_filename = os.path.basename(blob.name)
    processed_local_path = f"/tmp/processed_{local_filename}"
    processed_image.save(processed_local_path, format='JPEG')

    # Upload file đã xử lý lên folder destination trên GCS
    destination_blob_name = destination_prefix + local_filename
    destination_blob = bucket.blob(destination_blob_name)
    destination_blob.upload_from_filename(processed_local_path)
    print(f"Đã xử lý và upload {blob.name} lên {destination_blob_name}")

    # Xóa file tạm trên local
    os.remove(processed_local_path)

print("Quá trình ETL hoàn thành.")


/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/5361551308.jpg lên flickr_image_dataset/processed/5361551308.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/536156200.jpg lên flickr_image_dataset/processed/536156200.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/5362687336.jpg lên flickr_image_dataset/processed/5362687336.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/5363528971.jpg lên flickr_image_dataset/processed/5363528971.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/5363903246.jpg lên flickr_image_dataset/processed/5363903246.jpg
Đã xử lý và upload flickr_image_dataset/flickr30k_images/flickr30k_images/flickr30k_images/5364849864.jpg lên flickr_image_dataset/processed/5364849864.jpg
Đã xử lý 